In [ ]:
# importing all relevant modules and packages

import pandas as pd
import os
# import snscrape.modules.twitter as sntwitter
import json
import matplotlib.pyplot as plt
import numpy as np
import re
from PIL import Image
from os import path
import csv
# import datetime
# import random
# from datetime import date, timedelta
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# import tweepy
# from tweepy import OAuthHandler
import nltk

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving reviews.csv to reviews.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['reviews.csv'])) # File Name here should be replaced with name of CSV

<h2> 1. Data Cleaning </h2>

In [ ]:
# Data cleaning

# Apply a first round of text cleaning techniques
import re 
import string

def clean_text_round1(text):
    '''Make text lowercase, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) 
    # %s is string formatting into the square brackets and ''
    text = re.sub('\w*\d\w*', '', text) # \d meaning digits between a word \w* (or 0 word) so even 123abc will be removed entirely
    return text

# Removing emojis or taking only tweets using english alphabet
def remove_emojis(text):
    return text.encode('ascii', 'ignore').decode('ascii')

In [ ]:
# cleaning data
clean_text = []
for i in range(len(df["Reviews"].values.tolist())): 
    text = clean_text_round1(df["Reviews"].values.tolist()[i])
    text = remove_emojis(text)
    clean_text.append(text)
  
print(clean_text)

['terrible service from  staff at the counter  was just queuing to buy a lollipop the staff rudely told me what do you want  she expected me to\nstart from the last in the queue again when i had queued properly \n\nwhile the customer she was serving made a bigger purchase  very bias and rude staff  if  you are not willing to serve customers a lollipop dont put lollipops in your shop \nif you guys want candy go to candy empire at suntec  they have a wider variety of candy and service is much better', 'one of best place where we can visit to buy toys for all ages amazed with the collectiom they have', 'its very big but its very messy\n\nyou need to search for everything\n\nchildren heaven', 'loving it still find it in sg and love the promo\nso manh options that make you think to stay longer', 'worst customer service keep calling the hot line so many times but no one pick up same goes for email\n\ni bought this board game from toy r rus but with wrong game board  please advise i have emai

## 2 Classifying Tweets

After obtaining the crawled tweets, we classified them according to whether they were suicidal or not through the <b>*Zero*-Shot Classification Model</b>

* runtime has to be changed to GPU for this section

In [ ]:
# Packages for train/test dataset split
from sklearn.model_selection import train_test_split

In [ ]:
# this will take some time
!pip install datasets transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
## This will enable the colab notebook to use GPU
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

#should return cuda:0 if runtime is already GPU

'cuda:0'

In [ ]:
!pip install transformers
from transformers import pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.5 MB/s eta 0:00:00


In [ ]:
# Load the "sentiment prediction" model.
classifier = pipeline("zero-shot-classification", device=0)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
from sklearn.metrics import classification_report # to assess the quality of the model with the 100 hand annotated tweets

In [ ]:
df = pd.DataFrame()
df["clean_text"] = clean_text
df.head() # for checking 

,clean_text
0,terrible service from staff at the counter w...
1,one of best place where we can visit to buy to...
2,its very big but its very messy\n\nyou need to...
3,loving it still find it in sg and love the pro...
4,worst customer service keep calling the hot li...


<h3> 2.1 Zero-Shot Classification Model </h3>

In [ ]:

aspect_result = classifier(clean_text, candidate_labels=["positive", "negative"],)
aspect_result

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f45826614f0> was reported to be 10 (when accessing len(dataloader)), but 11 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f45826614f0> was reported to be 10 (when accessing len(dataloader)), but 12 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:640: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f45826614f0> was reported to be 10 (when accessing len(dataloader)), but 13 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataload

[{'sequence': 'terrible service from  staff at the counter  was just queuing to buy a lollipop the staff rudely told me what do you want  she expected me to\nstart from the last in the queue again when i had queued properly \n\nwhile the customer she was serving made a bigger purchase  very bias and rude staff  if  you are not willing to serve customers a lollipop dont put lollipops in your shop \nif you guys want candy go to candy empire at suntec  they have a wider variety of candy and service is much better',
  'labels': ['negative', 'positive'],
  'scores': [0.997086763381958, 0.0029132773634046316]},
 {'sequence': 'one of best place where we can visit to buy toys for all ages amazed with the collectiom they have',
  'labels': ['positive', 'negative'],
  'scores': [0.9844821095466614, 0.015517856925725937]},
 {'sequence': 'its very big but its very messy\n\nyou need to search for everything\n\nchildren heaven',
  'labels': ['negative', 'positive'],
  'scores': [0.9543688893318176, 

In [ ]:
y_aspect_pred = [result['labels'][np.argmax(result['scores'])] for result in aspect_result] # assigning the label with highst probability to that tweet
y_aspect_pred

['negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive']

In [ ]:
df['zero-shot'] = y_aspect_pred
df

,clean_text,zero-shot
0,terrible service from staff at the counter w...,negative
1,one of best place where we can visit to buy to...,positive
2,its very big but its very messy\n\nyou need to...,negative
3,loving it still find it in sg and love the pro...,positive
4,worst customer service keep calling the hot li...,negative
5,massive outlet with lots of toys in stock wen...,positive
6,use to be big but the downsized shop still doe...,negative
7,gave me contact numbers to call before coming ...,negative
8,no wonder they are out of business they dont e...,negative
9,this outlet is big in terms of their space are...,positive


In [ ]:
### We can do a confusion matrix and see the accuracy of this model if we have more data (and its labelled)